In [76]:
! pip install ydata-profiling
! pip install numpy==1.22.4

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip' command.


In [84]:
import pandas as pd
from datetime import date
from dateutil.relativedelta import relativedelta


In [40]:
# import data
df1_1 = pd.read_excel('证券2.xlsx', engine='openpyxl')
df1_2 = pd.read_excel('证券1.xlsx', engine='openpyxl')
df1_3 = pd.concat([df1_1, df1_2], ignore_index=True)
df1 = df1_3[df1_3['民营指数'] >= 1]


In [41]:
df2 = pd.read_excel('TRD_Mnth.xlsx', engine='openpyxl')
#df3 = pd.read_excel('TRD_Mnth.xlsx', engine='openpyxl')

In [42]:
df3 = df2.copy()

In [43]:
df2

,证券代码,交易月份,个股市值,个股月回报率,Unnamed: 4,Unnamed: 5
0,1,2010-01,67387912.64,-0.109561,NaN,NaN
1,1,2010-02,69716987.96,0.034562,NaN,NaN
2,1,2010-03,72046063.28,0.033408,NaN,NaN
3,1,2010-04,63847718.15,-0.113793,NaN,NaN
4,1,2010-05,54376145.17,-0.148346,NaN,NaN
...,...,...,...,...,...,...
573112,900957,2023-11,144486.00,0.037596,0.0,0.0
573113,900957,2023-12,144137.00,-0.002418,0.0,0.0
573114,900957,2024-01,143788.00,-0.002430,0.0,0.0
573115,900957,2024-02,155305.00,0.080101,0.0,0.0


In [44]:
df1.dtypes

统计截止日期       object
证券代码          int64
其他持股比例      float64
国有法人持股比例    float64
民营指数        float64
是否民营企业       object
dtype: object

In [45]:
df1['统计日期'] = pd.to_datetime(df1['统计截止日期'])
df2['统计日期'] = pd.to_datetime(df2['交易月份'])
df3['统计日期'] = pd.to_datetime(df3['交易月份'])

<ipython-input-45-435351aec7bd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['统计日期'] = pd.to_datetime(df1['统计截止日期'])


In [87]:
df1_sorted = df1.sort_values(by=['统计日期', '其他持股比例'], ascending=[True, False])
df1_sorted

,统计截止日期,证券代码,其他持股比例,国有法人持股比例,民营指数,是否民营企业,统计日期,分组
1167,2013-03-31,2703,94.92,0.00,inf,民营企业,2013-03-31,
451,2013-03-31,981,92.33,0.60,153.883333,民营企业,2013-03-31,
734,2013-03-31,2269,91.08,0.55,165.600000,民营企业,2013-03-31,
1115,2013-03-31,2651,90.92,0.18,505.111111,民营企业,2013-03-31,
1027,2013-03-31,2563,90.64,0.00,inf,民营企业,2013-03-31,
...,...,...,...,...,...,...,...,...
82710,2023-12-31,2407,18.60,1.57,11.847134,民营企业,2023-12-31,
154507,2023-12-31,600503,18.09,0.00,inf,民营企业,2023-12-31,
154470,2023-12-31,600354,16.26,14.09,1.154010,民营企业,2023-12-31,
82748,2023-12-31,2640,14.81,8.47,1.748524,民营企业,2023-12-31,


In [47]:
# grouped_returns = pd.DataFrame()

# # 9 三季报
# # 3 一季报
# # 6 半年报
# periods = {
#     9: (1, 4),
#     3: (5, 8),
#     6: (9, 12)
# }


In [48]:
report_months = {
    'third': 9,
    'first': 3,
    'half': 6
}
    

In [49]:
df1_sorted

,统计截止日期,证券代码,其他持股比例,国有法人持股比例,民营指数,是否民营企业,统计日期
1167,2013-03-31,2703,94.92,0.00,inf,民营企业,2013-03-31
451,2013-03-31,981,92.33,0.60,153.883333,民营企业,2013-03-31
734,2013-03-31,2269,91.08,0.55,165.600000,民营企业,2013-03-31
1115,2013-03-31,2651,90.92,0.18,505.111111,民营企业,2013-03-31
1027,2013-03-31,2563,90.64,0.00,inf,民营企业,2013-03-31
...,...,...,...,...,...,...,...
82710,2023-12-31,2407,18.60,1.57,11.847134,民营企业,2023-12-31
154507,2023-12-31,600503,18.09,0.00,inf,民营企业,2023-12-31
154470,2023-12-31,600354,16.26,14.09,1.154010,民营企业,2023-12-31
82748,2023-12-31,2640,14.81,8.47,1.748524,民营企业,2023-12-31


In [50]:
# add a new column in the df1
df1["分组"] = ""
df1
df_2013_3 = df1.loc[(df1['统计日期'].dt.year == 2013 )& (df1['统计日期'].dt.month == 3)]


<ipython-input-50-f4219f492461>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["分组"] = ""


In [66]:
df1

,统计截止日期,证券代码,其他持股比例,国有法人持股比例,民营指数,是否民营企业,统计日期,分组
0,2013-03-31,1,57.3500,1.81,31.685083,民营企业,2013-03-31,
1,2013-03-31,4,36.0100,0.00,inf,民营企业,2013-03-31,
2,2013-03-31,5,26.9300,3.64,7.398352,民营企业,2013-03-31,
4,2013-03-31,7,43.3800,0.00,inf,民营企业,2013-03-31,
5,2013-03-31,8,69.6900,0.00,inf,民营企业,2013-03-31,
...,...,...,...,...,...,...,...,...
154990,2023-12-31,871553,67.4700,0.00,inf,民营企业,2023-12-31,
154991,2023-12-31,872925,70.8403,0.00,inf,民营企业,2023-12-31,
154993,2023-12-31,873576,47.6400,30.91,1.541249,民营企业,2023-12-31,
154994,2023-12-31,873703,80.3100,0.00,inf,民营企业,2023-12-31,


In [88]:
#df1_copy = df1.copy()
df1_copy

,统计截止日期,证券代码,其他持股比例,国有法人持股比例,民营指数,是否民营企业,统计日期,分组
0,2013-03-31,1,57.3500,1.81,31.685083,民营企业,2013-03-31,
1,2013-03-31,4,36.0100,0.00,inf,民营企业,2013-03-31,
2,2013-03-31,5,26.9300,3.64,7.398352,民营企业,2013-03-31,
4,2013-03-31,7,43.3800,0.00,inf,民营企业,2013-03-31,
5,2013-03-31,8,69.6900,0.00,inf,民营企业,2013-03-31,
...,...,...,...,...,...,...,...,...
154990,2023-12-31,871553,67.4700,0.00,inf,民营企业,2023-12-31,
154991,2023-12-31,872925,70.8403,0.00,inf,民营企业,2023-12-31,
154993,2023-12-31,873576,47.6400,30.91,1.541249,民营企业,2023-12-31,
154994,2023-12-31,873703,80.3100,0.00,inf,民营企业,2023-12-31,


In [91]:

df1_group = pd.DataFrame()

df1_year_keymonth_size = 0

# 对公司进行分组 按照季度进行调仓
for year in df1['统计日期'].dt.year.unique():
    print('****** loop *******')
    print(year)
    
    
    for key, key_month in report_months.items():
        print('*****loop 季度*****')
        print(key)
        print(key_month)
         
        df1_year_keymonth = df1.loc[(df1['统计日期'].dt.year == year ) & (df1['统计日期'].dt.month == key_month)]
        
        df1_year_keymonth['分组'] = pd.qcut(df1_year_keymonth['其他持股比例'], 5, labels=False, duplicates='drop')
        
        
        df1_year_keymonth.info()
        
        df1_year_keymonth_size += len(df1_year_keymonth.index) 
        
        df1_group = df1_group.append(df1_year_keymonth, ignore_index=True)
        
        
df1_group

****** loop *******
2013
*****loop 季度*****
third
9
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1532 entries, 3046 to 84796
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   统计截止日期    1532 non-null   object        
 1   证券代码      1532 non-null   int64         
 2   其他持股比例    1532 non-null   float64       
 3   国有法人持股比例  1532 non-null   float64       
 4   民营指数      1532 non-null   float64       
 5   是否民营企业    1532 non-null   object        
 6   统计日期      1532 non-null   datetime64[ns]
 7   分组        1532 non-null   int64         
dtypes: datetime64[ns](1), float64(3), int64(2), object(2)
memory usage: 107.7+ KB
*****loop 季度*****
first
3
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1151 entries, 0 to 1521
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   统计截止日期    1151 non-null   object        
 1   证券代码      1151 non-nu

<ipython-input-91-385eae711461>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_year_keymonth['分组'] = pd.qcut(df1_year_keymonth['其他持股比例'], 5, labels=False)
<ipython-input-91-385eae711461>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_year_keymonth['分组'] = pd.qcut(df1_year_keymonth['其他持股比例'], 5, labels=False)
<ipython-input-91-385eae711461>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

*****loop 季度*****
first
3
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1731 entries, 12486 to 90649
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   统计截止日期    1731 non-null   object        
 1   证券代码      1731 non-null   int64         
 2   其他持股比例    1731 non-null   float64       
 3   国有法人持股比例  1731 non-null   float64       
 4   民营指数      1731 non-null   float64       
 5   是否民营企业    1731 non-null   object        
 6   统计日期      1731 non-null   datetime64[ns]
 7   分组        1731 non-null   int64         
dtypes: datetime64[ns](1), float64(3), int64(2), object(2)
memory usage: 121.7+ KB
*****loop 季度*****
half
6
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1840 entries, 14127 to 91711
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   统计截止日期    1840 non-null   object        
 1   证券代码      1840 non-null   int64         


<ipython-input-91-385eae711461>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_year_keymonth['分组'] = pd.qcut(df1_year_keymonth['其他持股比例'], 5, labels=False)
<ipython-input-91-385eae711461>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_year_keymonth['分组'] = pd.qcut(df1_year_keymonth['其他持股比例'], 5, labels=False)
<ipython-input-91-385eae711461>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1933 entries, 21060 to 96040
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   统计截止日期    1933 non-null   object        
 1   证券代码      1933 non-null   int64         
 2   其他持股比例    1933 non-null   float64       
 3   国有法人持股比例  1933 non-null   float64       
 4   民营指数      1933 non-null   float64       
 5   是否民营企业    1933 non-null   object        
 6   统计日期      1933 non-null   datetime64[ns]
 7   分组        1933 non-null   int64         
dtypes: datetime64[ns](1), float64(3), int64(2), object(2)
memory usage: 135.9+ KB
****** loop *******
2017
*****loop 季度*****
third
9
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2426 entries, 30470 to 102375
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   统计截止日期    2426 non-null   object        
 1   证券代码      2426 non-null   int64         

<ipython-input-91-385eae711461>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_year_keymonth['分组'] = pd.qcut(df1_year_keymonth['其他持股比例'], 5, labels=False)
<ipython-input-91-385eae711461>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_year_keymonth['分组'] = pd.qcut(df1_year_keymonth['其他持股比例'], 5, labels=False)
<ipython-input-91-385eae711461>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2983 entries, 54902 to 122066
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   统计截止日期    2983 non-null   object        
 1   证券代码      2983 non-null   int64         
 2   其他持股比例    2983 non-null   float64       
 3   国有法人持股比例  2983 non-null   float64       
 4   民营指数      2983 non-null   float64       
 5   是否民营企业    2983 non-null   object        
 6   统计日期      2983 non-null   datetime64[ns]
 7   分组        2983 non-null   int64         
dtypes: datetime64[ns](1), float64(3), int64(2), object(2)
memory usage: 209.7+ KB
*****loop 季度*****
first
3
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2783 entries, 50778 to 118245
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   统计截止日期    2783 non-null   object        
 1   证券代码      2783 non-null   int64         
 2   其他持股比例    2783 non

<ipython-input-91-385eae711461>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_year_keymonth['分组'] = pd.qcut(df1_year_keymonth['其他持股比例'], 5, labels=False)
<ipython-input-91-385eae711461>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_year_keymonth['分组'] = pd.qcut(df1_year_keymonth['其他持股比例'], 5, labels=False)
<ipython-input-91-385eae711461>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4038 entries, 80363 to 154299
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   统计截止日期    4038 non-null   object        
 1   证券代码      4038 non-null   int64         
 2   其他持股比例    4038 non-null   float64       
 3   国有法人持股比例  4038 non-null   float64       
 4   民营指数      4038 non-null   float64       
 5   是否民营企业    4038 non-null   object        
 6   统计日期      4038 non-null   datetime64[ns]
 7   分组        4038 non-null   int64         
dtypes: datetime64[ns](1), float64(3), int64(2), object(2)
memory usage: 283.9+ KB
*****loop 季度*****
first
3
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3927 entries, 76088 to 148006
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   统计截止日期    3927 non-null   object        
 1   证券代码      3927 non-null   int64         
 2   其他持股比例    3927 non

,统计截止日期,证券代码,其他持股比例,国有法人持股比例,民营指数,是否民营企业,统计日期,分组
0,2013-09-30,1,57.0600,1.8100,31.524862,民营企业,2013-09-30,2
1,2013-09-30,4,34.3400,0.0000,inf,民营企业,2013-09-30,0
2,2013-09-30,5,26.2200,3.6400,7.203297,民营企业,2013-09-30,0
3,2013-09-30,7,46.1200,0.4900,94.122449,民营企业,2013-09-30,1
4,2013-09-30,8,68.5400,0.0000,inf,民营企业,2013-09-30,3
...,...,...,...,...,...,...,...,...
84518,2023-06-30,873223,73.1996,0.6358,115.129915,民营企业,2023-06-30,4
84519,2023-06-30,873305,79.7865,0.0000,inf,民营企业,2023-06-30,4
84520,2023-06-30,873339,81.9562,0.0000,inf,民营企业,2023-06-30,4
84521,2023-06-30,873527,58.8700,0.0000,inf,民营企业,2023-06-30,2


In [92]:
df1_year_keymonth_size

84523

In [64]:
df1


,统计截止日期,证券代码,其他持股比例,国有法人持股比例,民营指数,是否民营企业,统计日期,分组
0,2013-03-31,1,57.3500,1.81,31.685083,民营企业,2013-03-31,
1,2013-03-31,4,36.0100,0.00,inf,民营企业,2013-03-31,
2,2013-03-31,5,26.9300,3.64,7.398352,民营企业,2013-03-31,
4,2013-03-31,7,43.3800,0.00,inf,民营企业,2013-03-31,
5,2013-03-31,8,69.6900,0.00,inf,民营企业,2013-03-31,
...,...,...,...,...,...,...,...,...
154990,2023-12-31,871553,67.4700,0.00,inf,民营企业,2023-12-31,
154991,2023-12-31,872925,70.8403,0.00,inf,民营企业,2023-12-31,
154993,2023-12-31,873576,47.6400,30.91,1.541249,民营企业,2023-12-31,
154994,2023-12-31,873703,80.3100,0.00,inf,民营企业,2023-12-31,


In [201]:
df3['DATE'] = df3['统计日期'].dt.date

df3['统计日期-4'] = df3['DATE'] - relativedelta(months=4)
df3.dtypes

证券代码                   int64
交易月份                  object
个股市值                 float64
个股月回报率               float64
Unnamed: 4           float64
Unnamed: 5           float64
统计日期          datetime64[ns]
DATE                  object
统计日期-4                object
dtype: object

In [202]:
df3

,证券代码,交易月份,个股市值,个股月回报率,Unnamed: 4,Unnamed: 5,统计日期,DATE,统计日期-4
0,1,2010-01,67387912.64,-0.109561,NaN,NaN,2010-01-01,2010-01-01,2009-09-01
1,1,2010-02,69716987.96,0.034562,NaN,NaN,2010-02-01,2010-02-01,2009-10-01
2,1,2010-03,72046063.28,0.033408,NaN,NaN,2010-03-01,2010-03-01,2009-11-01
3,1,2010-04,63847718.15,-0.113793,NaN,NaN,2010-04-01,2010-04-01,2009-12-01
4,1,2010-05,54376145.17,-0.148346,NaN,NaN,2010-05-01,2010-05-01,2010-01-01
...,...,...,...,...,...,...,...,...,...
573112,900957,2023-11,144486.00,0.037596,0.0,0.0,2023-11-01,2023-11-01,2023-07-01
573113,900957,2023-12,144137.00,-0.002418,0.0,0.0,2023-12-01,2023-12-01,2023-08-01
573114,900957,2024-01,143788.00,-0.002430,0.0,0.0,2024-01-01,2024-01-01,2023-09-01
573115,900957,2024-02,155305.00,0.080101,0.0,0.0,2024-02-01,2024-02-01,2023-10-01


In [203]:
# 进行加权平均计算月度回报率（分组过后+加权之后）
# 匹配时间节点上的公司市值
# 匹配相应时间段的月度回报率数据

# loop years
for year in df2['统计日期'].dt.year.unique():
    print('*************')
    print(year)
    current_year_data = df2[df2['统计日期'].dt.year == year] # type: ignore
    print(current_year_data.head())
    
    # loop 季报 调仓时候的市值 
    for key, key_month in report_months.items():
        print('**********')
        print(key)
        print(key_month*4/3)
        current_year_df_monthly = current_year_data[current_year_data['统计日期'].dt.month == key_month*4/3]
        print(current_year_df_monthly.head())
        #计算权重
        
        current_year_df_monthly['季报日期']
        
        
        
        
        
        
        

*************
2010
   证券代码     交易月份         个股市值    个股月回报率  Unnamed: 4  Unnamed: 5       统计日期
0     1  2010-01  67387912.64 -0.109561         NaN         NaN 2010-01-01
1     1  2010-02  69716987.96  0.034562         NaN         NaN 2010-02-01
2     1  2010-03  72046063.28  0.033408         NaN         NaN 2010-03-01
3     1  2010-04  63847718.15 -0.113793         NaN         NaN 2010-04-01
4     1  2010-05  54376145.17 -0.148346         NaN         NaN 2010-05-01
**********
third
12.0
     证券代码     交易月份         个股市值    个股月回报率  Unnamed: 4  Unnamed: 5       统计日期
9       1  2010-12  55028367.30 -0.036608         NaN         NaN 2010-12-01
180     2  2010-12  79571694.05  0.011070         NaN         NaN 2010-12-01
344     4  2010-12   1011079.28 -0.063764         NaN         NaN 2010-12-01
508     5  2010-12   3355604.34 -0.046753         NaN         NaN 2010-12-01
664     6  2010-12   5302601.00  0.013081         NaN         NaN 2010-12-01
**********
first
4.0
     证券代码     交易月份        

In [165]:
# join df1_group and df2

#df1_group.set_index('证券代码').join(df2.set_index('证券代码'), on=['证券代码'], lsuffix='_group', rsuffix='_df2')

#df1_group.set_index(['统计日期','证券代码']).join(df2.set_index(['统计日期','证券代码']), on=['统计日期','证券代码'],how = 'outer', lsuffix='_group', rsuffix='_df2')

#df1_group_join  = df1_group.set_index(['证券代码']).join(df2.set_index(['证券代码']), on=['证券代码'],how = 'inner', lsuffix='_group', rsuffix='_df2')

#df1_group_join.reset_index()


In [166]:

#df1_group_join['统计日期_df2_str'] = str(df1_group_join['统计日期_df2'])

#df1_group_join['统计日期_df2_str'] = df1_group_join['统计日期_df2'].dt.strftime('%Y-%m-%d')




In [167]:
#df1_group_join


In [168]:
#df1_group_join.dtypes

In [162]:
#df1_final = df1_group_join[df1_group_join['统计截止日期'] == df1_group_join['统计日期_df2_str']]

In [191]:
from datetime import date
from dateutil.relativedelta import relativedelta

current_date = date.today()

print(current_date)

future_date = current_date - relativedelta(months=4)

print(future_date)

2024-05-13
2024-01-13


In [195]:
dt = pd.to_datetime('2024-04-01')
dt

dt1 = dt - relativedelta(months=4)
dt1

Timestamp('2023-12-01 00:00:00')

In [ ]:
# 月度回报率数据

In [51]:

for month, (start_month, end_month) in periods.items():
    # 筛选出分组月份的数据
    df1_monthly = df1_sorted[df1_sorted['统计日期'].dt.month == month]

    # 对每一年的数据进行操作
    for year in df1_monthly['统计日期'].dt.year.unique():
        # 获取当前年份和上一年的数据
        current_year_data = df1_monthly[df1_monthly['统计日期'].dt.year == year]
        previous_year_data = df1_monthly[df1_monthly['统计日期'].dt.year == year - 1]

        # 对当前年份的数据进行分组
        current_year_data['分组'] = pd.qcut(current_year_data['其他持股比例'], 5, labels=False, duplicates='drop')

        # 遍历每个分组
        for group in range(5):
            # 获取上一年同一分组的证券代码
            previous_group_codes = previous_year_data[previous_year_data['证券代码'].isin(current_year_data[current_year_data['分组'] == group]['证券代码'])]['证券代码']

            # 计算加权回报率
            for month in range(start_month, end_month + 1):
                # 获取对应月份的数据
                df2_monthly = df2[(df2['统计日期'].dt.year == year) & (df2['统计日期'].dt.month == month)]

                # 筛选出属于当前分组的股票
                df2_group = df2_monthly[df2_monthly['证券代码'].isin(previous_group_codes)]

                # 计算加权回报率
                total_market_value = df2_group['个股市值'].sum()
                df2_group['加权回报率'] = df2_group['个股市值'] / total_market_value * df2_group['个股月回报率']
                weighted_return = df2_group['加权回报率'].sum()

                # 将结果存储到grouped_returns中
                new_data = pd.DataFrame({
                    '年份': [year],
                    '月份': [month],
                    '分组': [group],
                    '加权回报率': [weighted_return]
                })

                grouped_returns = pd.concat([grouped_returns, new_data], ignore_index=True)
print(grouped_returns.head())
grouped_returns.to_excel('计算.xlsx', index=False)